In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = ""
# print(os.environ.get('OPENAI_API_KEY'))


In [2]:
%load_ext autoreload
%autoreload all

from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))

llm_result = llm.generate(["Tell me a joke", "Tell me a poem"]*15)
llm_result




Funky Feet Socks.


LLMResult(generations=[[Generation(text='\n\nQ: What did the fish say when it hit the wall?\nA: Dam!', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nRoses are red,\nViolets are blue,\nSugar is sweet\nAnd so are you.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text="\n\nQ: What kind of key can't open a door?\nA: A turkey!", generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe fog comes\non little cat feet.\nIt sits looking\nover harbor and city\non silent haunches\nand then moves on.\n\n-Carl Sandburg', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nQ: What did the Zen Buddist say to the hotdog vendor?\nA: Make me one with everything.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nRoses are red\nViolets are blue\nSugar is sweet\nAnd so are you', generation_info={'finish_reason': 'stop', 'logprobs': None}

In [ ]:
from langchain.docstore.document import Document
from langchain.document_loaders import ApifyDatasetLoader

import os
os.environ["APIFY_API_TOKEN"] = "TODO"

loader = ApifyDatasetLoader(
    dataset_id="FC5s0pHUeicpiXhBh",
    dataset_mapping_function=lambda dataset_item: Document(
        page_content=dataset_item["description"], metadata={
            "source": dataset_item["url"],
            "title": dataset_item["title"],
            "time": dataset_item["time"],
            "location": dataset_item["location"],
        }
    ),
)

data = loader.load()
data


[Document(page_content='DetailsIt\'s almost time for the Five Boro Bike Tour. We can\'t wait to ride with you!\nExciting news - Brompton is sponsoring our After Party at a rooftop terrace in DUMBO!\nImportant\nThe Five Boro Bike Tour is an event organized by Bike NYC with 30000 riders. This meetup organizes a small group of Bromptons within the larger ride. You must register and pay for the ride with Bike NYC.\nWave 4 Riders\n(and those in Waves 1-3 who want to start with Wave 4)\nPlease check the map and "How to find us" section for details on how to meet for the Wave 4 start.\nWave 5-6 Riders\nYou won\'t be able to start in Wave 4. Please post in the comments if you want to find other Brompton riders in Waves 5-6.\nBrompton-only Afterparty in DUMBO\nJoin us on a rooftop terrace in DUMBO after the ride to celebrate! Details will be sent out the weekend of the ride via meetup direct message. If you don\'t receive the information, please message Host Dana as some people people\'s privac

## Question answering

In [7]:
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator().from_loaders([loader])

query = "What some of the tech events that take place on May 15?"
result = index.query_with_sources(query)

result

# print(result["answer"])
# print(result["sources"])


Using embedded DuckDB without persistence: data will be transient


{'question': 'What some of the tech events that take place on May 15?',
 'answer': " The tech events taking place on May 15 include NYC Tech Happy Hour, a tech event hosted by MariaDB Conference, NYC High-Tech Deep-Tech Matrix's GeekMixer: Byte & Mingle, and a Claris Product Updates virtual meetup.\n",
 'sources': 'https://www.meetup.com/new-york-tech-happy-hour/events/293226173/\nhttps://www.meetup.com/meetup-group-mariadbconference/events/292667801/\nhttps://www.meetup.com/tech-house-nyc/events/292774360/\nhttps://www.meetup.com/nynjclarisfilemaker/events/292954426/'}

In [22]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

# page_content=dataset_item["description"], metadata={
#     "source": dataset_item["url"],
#     "title": dataset_item["title"],
#     "time": dataset_item["time"],
#     "location": dataset_item["location"],
# }

metadata_field_info=[
    AttributeInfo(
        name="time",
        description="Time of the event", 
        type="string", 
    ),
    AttributeInfo(
        name="location",
        description="Event location", 
        type="string", 
    ),
    AttributeInfo(
        name="title",
        description="The event name",
        type="string", 
    ),
]

# document_content_description = "Event description"
# llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(llm, index.vectorstore, document_content_description, metadata_field_info, verbose=True)


NameError: name 'v_args' is not defined

In [ ]:
from langchain import PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

template = """

"""

examples = [
    {""}
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1
)

# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")
